In [85]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [ ]:
!unzip -q /kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
!unzip -q /kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip

In [ ]:
os.listdir('/kaggle/working/train')

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img
img = load_img('/kaggle/working/train/dog.5606.jpg')
plt.imshow(img)

In [ ]:
train = pd.DataFrame({'path':os.listdir('/kaggle/working/train')})
train['path'] = '/kaggle/working/train/'+train['path']
train['label'] = train['path'].apply(lambda x:x.split('/')[-1].split('.')[0])
train.head()

In [ ]:
test = pd.DataFrame({'path':os.listdir('/kaggle/working/test')})
test['path'] = '/kaggle/working/test/'+test['path']
test.head()

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid = train_test_split(train, test_size=0.1, stratify=train['label'], random_state=5)
print('훈련용', x_train['label'].value_counts())
print('검증용', x_valid['label'].value_counts())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, vertical_flip=True, fill_mode='nearset')
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_datagen.flow_from_dataframe(x_train, x_col='path', y_col='label',
                                                   batch_size=16, target_size=(300, 300))
valid_generator = train_datagen.flow_from_dataframe(x_valid, x_col='path', y_col='label',
                                                   batch_size=16, target_size=(300, 300))
test_generator = train_datagen.flow_from_dataframe(test, x_col='path',
                                                   class_mode=None, shuffle=False,
                                                   batch_size=16, target_size=(300, 300))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.models import Sequential

In [ ]:
model = Sequential()
model.add(Conv2D(128, (3,3), activation='relu', input_shape=(300,300,3)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(32, (3, 3), activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(16, (3, 3), activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [ ]:
 model.summary()

In [ ]:
model.fit_generator(train_generator, validation_data=valid_generator, epochs=1)

In [ ]:
from tensorflow.keras.applications import VGG16

transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
transfer_model.trainable=False

transfer_model.summary()

In [ ]:
model2 = Sequential()
model2.add(transfer_model)
model2.add(Flatten())
model2.add(Dense(64, activation='relu'))
model2.add(Dense(2, activation='softmax'))
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model2.summary()

In [ ]:
model2.fit_generator(train_generator, validation_data=valid_generator, epochs=1)

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2
transfer_model2 = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
transfer_model2.transable=False

In [ ]:
model3 = Sequential()
model3.add(transfer_model2)
model3.add(Flatten())
model3.add(Dense(64, activation='relu'))
model3.add(Dense(2, activation='softmax'))
model3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
model3.summary()

In [ ]:
model3.fit_generator(train_generator, validation_data=valid_generator, epochs=1)

In [ ]:
result = model3.predict_generator(test_generator, verbose=1, workers=2)

In [ ]:
sub = pd.read_csv('/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv')
sub.head()

In [ ]:
sub['label'] = result[:,1]
sub['id']=test_generator.filenames
sub['id']=sub['id'].apply(lambda x:x.split('/')[-1].split('.')[0])
sub.head()

In [ ]:
sub.to_csv('sub.csv', index=False)